In [16]:
from mmengine.registry import OPTIMIZERS
from mmengine.config import Config
import torch

cfg=Config.fromfile('/volume/opemmmlab/myengine/Advanced-tutorials/Config/tmp/config_sgd.py')
model=torch.nn.Conv2d(1,1,1)
print(cfg)

Config (path: /volume/opemmmlab/myengine/Advanced-tutorials/Config/tmp/config_sgd.py): {'optimizer': {'type': 'SGD', 'lr': 0.1, 'momentum': 0.9, 'weight_decay': 0.0001}}


In [17]:
cfg.optimizer.params=model.parameters()
print(cfg)

Config (path: /volume/opemmmlab/myengine/Advanced-tutorials/Config/tmp/config_sgd.py): {'optimizer': {'type': 'SGD', 'lr': 0.1, 'momentum': 0.9, 'weight_decay': 0.0001, 'params': <generator object Module.parameters at 0x7ff78da85000>}}


In [18]:
#在这jupyter里就会报错，但是py文件里不会
optim=OPTIMIZERS.build(cfg.optimizer)
print(optim)

KeyError: 'SGD is not in the optimizer registry. Please check whether the value of `SGD` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'

##### 继承保留字
1. _base_,多个文件需要列表形式，且需要没有重合字段

##### 修改继承字段
1. 如某个字段继承自父文件，但是需要修改，只需要重新定义，如果该字段是嵌套字典，则只需要重新定义需要修改的子字段即可；但是如果该字段不是字典，重新定义则会完全覆盖

##### 删除key
1. 使用关键字_delete_,可以保证如果新定义的字段中没有的key，会从继承字段中全部删除（适用于字典字段）

引用继承文件中的变量
1. 使用a={{_base_.key}}的方式引用，适用于py，yaml，json文件，但如果想要修改引用的变量，不能使用a['key']=xxx的方式
2. 使用a=_base_.key的方式，只适用py文件，且修改方式为a.key=xxx

导出配置文件 \
使用cfg.dump(path)接口

命令行传参修改配置文件
1. 我的本意是想要在调用Config.fromfile()接口之前，修改某个字段，该字段为一个中间变量，让其他所有用这个字段的地方一起修改，因为Config.fromfile()接口会自动将所有用到的变量替换为字段.
2. 但实际上这个办法也是在Config.fromfile()接口之后修改的，并不能够达到想要的效果
3. 不过他确实可以通过命令行来动态修改配置文件的字段，用法即传入--cfg-options参数，后面接要修改的字段和修改后的值，如--cfg-options optimizer.lr=0.1 --cfg-options model.in_channels="[1, 1, 1]"
4. 如上他可以多次传入，且修改列表等必须要用引号引起来，另外要求读取配置文件的脚本需要argsparse添加action为DictAction的--cfg-options参数.

环境变量修改配置文件
1. 这个方法是在调用Config.fromfile()接口之前，修改某个字段，该字段为一个中间变量，让其他所有用这个字段的地方一起修改，可以实现我的需求
2. 用法是，{{$修改的变量:不传入时的默认值}}，如果是修改字符串，那么这样写data_root="{{$DATA_ROOT:/data/}}"
如果修改的不是字符串，则这样写data_root={{"$classes:80"}}

导入自定义python模块
这个功能是用在已经开发好的项目中，注册了新的模块，我们想在配置文件中通过dict的方式使用该模块的情况下
使用配置文件的保留字段custom_imports

跨项目继承和获取配置文件
使用mmdet::的形式来指定继承或者获取哪个项目下的配置文件
获取从mmengine.hub导入get_config和get_model接口